#### 6. Ejercicio: Análisis de sentimientos en español

Análisis de sentimientos de los tweets en español sobre aerolineas: Analizar cómo los viajeros expresaron sus sentimientos.


In [8]:
# Librerias
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [3]:
df = pd.read_csv('tweets_public.csv', encoding='utf-8', index_col='tweet_id')
df.head(5)

,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
tweet_id,,,,,,,,,
926419989107798016,neutral,False,0,0,Trabajar en #Ryanair como #TMA: https://t.co/r...,NaN,Fri Nov 03 12:05:12 +0000 2017,NaN,Madrid
934854385577943041,neutral,True,0,0,@Iberia @FIONAFERRER Cuando gusten en Cancún s...,NaN,Sun Nov 26 18:40:28 +0000 2017,NaN,Mexico City
945318406441635840,negative,False,0,0,Sabiais que @Iberia te trata muy bien en santi...,NaN,Mon Dec 25 15:40:45 +0000 2017,NaN,Madrid
927540721296568320,negative,False,0,0,NUNCA NUNCA NUNCA pidáis el café de Ryanair.\n...,NaN,Mon Nov 06 14:18:35 +0000 2017,NaN,Pacific Time (US & Canada)
947965901332197376,positive,True,0,0,@cris_tortu @dakar @Iberia @Mitsubishi_ES @BFG...,NaN,Mon Jan 01 23:00:57 +0000 2018,NaN,Buenos Aires


In [5]:
# Feedback Value count
df.airline_sentiment.value_counts()

negative    3769
neutral     2609
positive    1489
Name: airline_sentiment, dtype: int64

#### Preprocesamientos y representación vectorial

In [11]:
import spacy
import string
from spacy.lang.es.stop_words import STOP_WORDS
from spacy.lang.es import Spanish

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
stop_words=""

# Load English tokenizer, tagger, parser, NER and word vectors
parser = Spanish()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [12]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
bow_vector

CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7fa96eccc488>)

In [13]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

#### Partición de datos

In [15]:
from sklearn.model_selection import train_test_split

X = df['text'] # the features we want to analyze
ylabels = df['airline_sentiment'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.5)


#### Generación del modelo de clasificación

In [16]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
modelLR = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([('preprocessing', bow_vector),
                 ('regression-ML', modelLR)])

# model generation
pipe.fit(X_train,y_train)

/home/jelbo/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(steps=[('preprocessing',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7fa96eccc488>)),
                ('regression-ML', LogisticRegression())])

#### Evaluación del modelo 

In [25]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)
print(predicted)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted, average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted, average='weighted'))

['neutral' 'negative' 'negative' ... 'negative' 'neutral' 'neutral']
Logistic Regression Accuracy: 0.5849008642602949
Logistic Regression Precision: 0.5762942034540115
Logistic Regression Recall: 0.5849008642602949


In [26]:
#Evaluación del rendimiento del clasificador
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, predicted)
print(confusion_matrix)
#Print de la matriz de confusión
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted))

[[1408  384  105]
 [ 483  636  173]
 [ 175  313  257]]
              precision    recall  f1-score   support

    negative       0.68      0.74      0.71      1897
     neutral       0.48      0.49      0.48      1292
    positive       0.48      0.34      0.40       745

    accuracy                           0.58      3934
   macro avg       0.55      0.53      0.53      3934
weighted avg       0.58      0.58      0.58      3934



In [42]:
def printNMostInformative(vectorizer, model, N):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(model.coef_[0], feature_names))
    topClass1 = coefs_with_fns[:N]
    topClass2 = coefs_with_fns[:-(N + 1):-1]
    topClass3 = coefs_with_fns[0:0]   #Esos valores determinan el neutral
    #Tengo que definir que rango val1:val2 es neutral
    print("Class 1 best: ")
    for feat in topClass1:
        print(feat)
    print("Class 2 best: ")
    for feat in topClass2:
        print(feat)
    print("Class 3 best: ")
    for feat in topClass3:
        print(feat)

In [43]:
printNMostInformative(bow_vector, modelLR, 20)

Class 1 best: 
(-0.9569317849042004, 'felicidades')
(-0.9498548765582929, 'gracias')
(-0.890321412161182, 'foto')
(-0.8493104605352927, 'reconocerá')
(-0.8242900485947552, 'holaargentina')
(-0.8205271783703655, 'feliz')
(-0.7710217714504983, 'necesita')
(-0.7387460254054002, 'quiero')
(-0.7202436831537699, 'mira')
(-0.6833973489516223, 'reconocer')
(-0.682701798761811, 'encanta')
(-0.6787189503724856, 'salido')
(-0.6693770810064339, 't')
(-0.6660728521272005, 'mejor')
(-0.6490279107430258, '@fjlopezm')
(-0.6461405210519526, 'b')
(-0.6355517755981822, 'barato')
(-0.6340962002410947, 'desconvocan')
(-0.6329973385349146, 'sindicatos')
(-0.6276880908885452, 'buen')
Class 2 best: 
(1.3450082499148317, 'retraso')
(1.0509972991338727, 'vergüenza')
(1.0113276206561146, '@supermanlopezn')
(0.9822566191217572, 'normal')
(0.9723167631232462, 'ninguna')
(0.8807567127164401, 'puto')
(0.8799672448994099, 'retrasados')
(0.8787598804663005, 'peor')
(0.868624193271852, 'caro')
(0.8663845249849166, 'mie